<a href="https://colab.research.google.com/github/neversettlejay/mineai_cote_gold/blob/no_validation/MineAI_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MineAI - Optimizing Open-Pit Blast Performance with Machine Learning

## Project Overview

This project focuses on using algorithms and available data to rate the effectiveness of open pit blasts. The goal is to suggest optimal parameters that can improve drill and blast efficiency, as well as reduce the deviation between planned and actual drilling results (e.g., drill hole deviations, blast outcomes, etc.). Through data analysis and predictive modeling, the project aims to enhance mining operations by optimizing blast designs, improving fragmentation, and reducing unplanned costs due to inefficiencies in drilling and blasting.


## Team Members

* Manav Chaudhary - Mining Engineer
* Shrey Patel - Machine Learning Engineer
* Soham Salakhana - Mining Engineer
* Shakil Kalvatar - Mining Engineer
* Jay Rathod - Software Engineer


## Data and Methodology

This project will leverage:

* Historical data from past drill and blast operations
* Sensor data from the mine, including real-time measurements of blast outcomes and drill hole characteristics
* Statistical methods, machine learning algorithms, and optimization techniques to rate blast performance
* Suggestions for optimizing parameters like drill depth, spacing, and blast charge amounts


## Key Objectives

* Rate the success of each blast based on fragmentation size, efficiency, and deviation from the planned design.
* Use machine learning algorithms to identify patterns in the data and suggest improvements to drill and blast parameters.
* Propose changes to the drilling process to reduce deviation and optimize efficiency, ultimately improving cost and time savings.

# Installing Dependencies

This section outlines the necessary steps to install the required dependencies for this project.

We will be using the following libraries:

* **pdf2image:**  Converts PDF files to images for analysis.
* **imagehash:**  Generates perceptual hashes of images for comparison and similarity detection.
* **poppler-utils:** Provides utilities for working with PDF files, including rendering and conversion.
* **opencv-python:**  Open Source Computer Vision Library for image and video processing.

In [ ]:
# Installing dependencies

!pip install pdf2image
!pip install imagehash
!apt-get install -y poppler-utils
# !pip uninstall -y opencv-python opencv-python-headless
!nvidia-smi
!pip install opencv-python


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/packaging/markers.py", line 179, in _eval_op
    spec = Specifier("".join([op.serialize(), rhs]))
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/packaging/specifiers.py", line 237, in __init__
    raise InvalidSpecifier(f"Invalid specifier: '{spec}'")
pip._vendor.packaging.specifiers.InvalidSpecifier: Invalid specifier: '==dev'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/

In [ ]:
# To use GPU
# Remove any existing OpenCV installations
# !pip uninstall -y opencv-python opencv-python-headless

# # Install dependencies
# !apt-get update && apt-get install -y \
#     build-essential cmake git unzip pkg-config \
#     libjpeg-dev libpng-dev libtiff-dev \
#     libavcodec-dev libavformat-dev libswscale-dev \
#     libv4l-dev libxvidcore-dev libx264-dev \
#     libgtk-3-dev libcanberra-gtk* \
#     libatlas-base-dev gfortran python3-dev \
#     libdc1394-22-dev libopenexr-dev libgstreamer-plugins-base1.0-dev \
#     libgstreamer1.0-dev libgphoto2-dev libeigen3-dev

# # Clone OpenCV repositories
# !git clone https://github.com/opencv/opencv.git
# !git clone https://github.com/opencv/opencv_contrib.git

# # Create a build directory
# !mkdir -p opencv/build
# %cd opencv/build

# # Configure the build with CUDA support
# !cmake -D CMAKE_BUILD_TYPE=RELEASE \
#        -D CMAKE_INSTALL_PREFIX=/usr/local \
#        -D OPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules \
#        -D WITH_CUDA=ON \
#        -D ENABLE_FAST_MATH=ON \
#        -D CUDA_FAST_MATH=ON \
#        -D WITH_CUDNN=ON \
#        -D OPENCV_DNN_CUDA=ON \
#        -D BUILD_opencv_python3=ON ..

# # Build OpenCV (this might take a while)
# !make -j$(nproc)

# # Install the newly built OpenCV
# !make install
# !ldconfig


In [ ]:
# TO check whether GPU can be used
import cv2
print(cv2.cuda.getCudaEnabledDeviceCount())  # Should print a positive integer


0


In [ ]:
from google.colab import drive
drive.flush_and_unmount()


#importing the given data from google drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the file in Google Drive
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
#extract images from pdf
import os
import shutil
!pip install pdf2image
!apt-get install -y poppler-utils
from pdf2image import convert_from_path
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Base directory path in Google Drive
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'

# Define the main directories
main_folders = ["data"]

# --- CREATE CORRECT STRUCTURE AND EXTRACT IMAGES ---
for main_folder in main_folders:
    main_folder_path = os.path.join(parent_directory_path, main_folder)

    for subfolder in os.listdir(main_folder_path):
        subfolder_path = os.path.join(main_folder_path, subfolder)

        # Check if it's a directory
        if os.path.isdir(subfolder_path):
            # Create a 'report' folder if it doesn't exist
            report_folder = os.path.join(subfolder_path, 'report')
            os.makedirs(report_folder, exist_ok=True)

            # Find all PDF files in the subfolder
            pdf_files = [f for f in os.listdir(subfolder_path) if f.endswith('.pdf')]

            for pdf_file in pdf_files:
                pdf_path = os.path.join(subfolder_path, pdf_file)

                # Create a subfolder inside 'report' based on the PDF name
                pdf_name = os.path.splitext(pdf_file)[0]
                pdf_report_folder = os.path.join(report_folder, pdf_name)
                os.makedirs(pdf_report_folder, exist_ok=True)

                # Convert PDF to images
                try:
                    images = convert_from_path(pdf_path)
                    for i, image in enumerate(images):
                        image_path = os.path.join(pdf_report_folder, f'image_{i + 1}.jpg')
                        image.save(image_path, 'JPEG')

                    print(f"Extracted images from {pdf_file} saved in {pdf_report_folder}")

                except Exception as e:
                    print(f"Error processing {pdf_file}: {e}")

In [ ]:
#to list directories
import os

# Assuming you've already mounted your Google Drive
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'
data_folder = os.path.join(parent_directory_path, 'data')

# List subdirectories
for subdirectory in os.listdir(data_folder):
    subdirectory_path = os.path.join(data_folder, subdirectory)
    if os.path.isdir(subdirectory_path):
        print(subdirectory_path)

/content/drive/My Drive/pour_la_hackathon/MineAI/data/328_109
/content/drive/My Drive/pour_la_hackathon/MineAI/data/340_102
/content/drive/My Drive/pour_la_hackathon/MineAI/data/352_103
/content/drive/My Drive/pour_la_hackathon/MineAI/data/352_108
/content/drive/My Drive/pour_la_hackathon/MineAI/data/352_121
/content/drive/My Drive/pour_la_hackathon/MineAI/data/364_303
/content/drive/My Drive/pour_la_hackathon/MineAI/data/352_312
/content/drive/My Drive/pour_la_hackathon/MineAI/data/352_138
/content/drive/My Drive/pour_la_hackathon/MineAI/data/352_137
/content/drive/My Drive/pour_la_hackathon/MineAI/data/352_132
/content/drive/My Drive/pour_la_hackathon/MineAI/data/352_131
/content/drive/My Drive/pour_la_hackathon/MineAI/data/364_105


In [ ]:
#crop video (moving object)
import os
import logging
import cv2
!pip install ultralytics
from ultralytics import YOLO

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define directories
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'
video_folder = os.path.join(parent_directory_path, 'data')
output_folder = os.path.join(parent_directory_path, 'processed_videos')

# Subfolders to process
subfolders = ['C1_328_109', 'C1_340_102', 'C1_352_103', 'C1_352_108', 'C1_352_137', 'C1_352_132',
              'C1_352_131', 'C1_352_121', 'C1_364_303', 'C1_364_105', 'C1_352_312', 'C1_352_138']

# Supported video extensions
video_extensions = ['.mp4', '.MP4', '.avi', '.AVI', '.mov', '.MOV', '.mkv', '.MKV']

# Ensure output directory structure exists
for subfolder in subfolders:
    os.makedirs(os.path.join(output_folder, subfolder), exist_ok=True)

# Initialize YOLOv8 model
model = YOLO("yolov8s.pt")  # Load the pretrained model

# Define helper functions
def box_center(coords):
    left, top, right, bottom = coords
    return [(left + right) / 2, (top + bottom) / 2]

def adjust_box_size(coords, box_width, box_height):
    center_x, center_y = box_center(coords)
    return [
        center_x - box_width / 2, center_y - box_height / 2,
        center_x + box_width / 2, center_y + box_height / 2
    ]

def adjust_boundaries(coords, screen):
    left, top, right, bottom = coords
    width, height = screen
    if left < 0:
        right -= left
        left = 0
    if top < 0:
        bottom -= top
        top = 0
    if right > width:
        left -= (right - width)
        right = width
    if bottom > height:
        top -= (bottom - height)
        bottom = height
    return [round(left), round(top), round(right), round(bottom)]

def check_video_empty(vid_capture):
    """Check if a video is empty or corrupt."""
    if vid_capture.isOpened() == False:
        return True
    ret, frame = vid_capture.read()
    if not ret or frame is None:
        return True
    return False

def is_steady_frame(frame, last_frame, threshold=5):
    """Check if the current frame is steady (no significant changes)."""
    if last_frame is None:
        return False
    # Compute the difference between the current and the previous frame
    diff = cv2.absdiff(frame, last_frame)
    diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    non_zero_count = cv2.countNonZero(diff_gray)
    if non_zero_count < threshold:
        return True  # Steady frame
    return False

# Process videos
for subfolder in subfolders:
    input_path = os.path.join(video_folder, subfolder)
    output_path = os.path.join(output_folder, subfolder)

    for file_name in os.listdir(input_path):
        if os.path.splitext(file_name)[1] in video_extensions:
            input_video = os.path.join(input_path, file_name)
            output_video = os.path.join(output_path, f"processed_{subfolder}.mp4")

            logger.info(f"Processing video: {input_video}")

            # Open input video
            vid_capture = cv2.VideoCapture(input_video)
            fps = int(vid_capture.get(cv2.CAP_PROP_FPS))
            width = int(vid_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(vid_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

            # Check if the video is empty or corrupt
            if check_video_empty(vid_capture):
                logger.warning(f"Skipping empty or corrupt video: {input_video}")
                continue

            # Initialize cropping box
            box_width, box_height = 500, 500
            default_coords = [100, 100, 600, 600]  # Default box
            last_box_coords = default_coords
            last_frame = None

            # Open output video stream
            output_writer = cv2.VideoWriter(
                output_video,
                cv2.VideoWriter_fourcc(*'mp4v'),
                fps,
                (box_width, box_height)
            )

            # Process video frames
            frame_counter = 0
            steady_frame_counter = 0  # To track how many steady frames
            min_frames = 5 * fps  # Ensure the video is at least 5 seconds long
            while True:
                ret, frame = vid_capture.read()
                if not ret:
                    break

                # Check if the frame is steady
                if is_steady_frame(frame, last_frame):
                    steady_frame_counter += 1
                else:
                    steady_frame_counter = 0  # Reset if motion is detected

                # YOLO detection
                results = model.predict(source=frame, conf=0.5, iou=0.1)
                boxes = results[0].boxes

                if boxes:
                    # Use the first detected box
                    box = boxes[0].xyxy[0].cpu().numpy().astype(int)
                    last_box_coords = adjust_box_size(box, box_width, box_height)
                    last_box_coords = adjust_boundaries(last_box_coords, [width, height])

                # Use the last known cropping box
                box_left, box_top, box_right, box_bottom = last_box_coords

                # Crop and write frame
                cropped_frame = frame[box_top:box_bottom, box_left:box_right]
                output_writer.write(cropped_frame)

                # Update last frame for steady check
                last_frame = frame

                frame_counter += 1

            # Ensure video has sufficient length
            if frame_counter < min_frames:
                logger.info(f"Padding {min_frames - frame_counter} frames to video {output_video}")
                padding_frame = cropped_frame.copy()
                for _ in range(min_frames - frame_counter):
                    output_writer.write(padding_frame)

            # Log if there were many steady frames
            if steady_frame_counter > 50:
                logger.info(f"Detected {steady_frame_counter} steady frames in {input_video}")

            # Release resources
            vid_capture.release()
            output_writer.release()
            logger.info(f"Saved processed video to {output_video}")


In [ ]:
#extract frames from video every 300 ms
import os
import cv2
import logging
import numpy as np

# Directories
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'
video_folder = os.path.join(parent_directory_path, 'data')  # Path to the "data" folder
extracted_frames_dir = os.path.join(parent_directory_path, 'extracted_frames')

# Ensure output directory exists
os.makedirs(extracted_frames_dir, exist_ok=True)

# Logging configuration
logging.basicConfig(
    filename=os.path.join(parent_directory_path, 'frame_extraction.log'),
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Common video file extensions
video_extensions = ['.mp4', '.MP4', '.avi', '.AVI', '.mov', '.MOV', '.mkv', '.MKV']

def extract_frames(video_path, output_dir, frame_step=10):
    """
    Extract frames from a video and save them based on timestamps in milliseconds.

    Args:
        video_path (str): Path to the video file.
        output_dir (str): Directory to save the frames.
        frame_step (int): Interval to save frames (e.g., every nth frame).
    """
    video_name = os.path.basename(video_path).split('.')[0]
    video_frames_dir = os.path.join(output_dir, video_name)
    os.makedirs(video_frames_dir, exist_ok=True)  # Create a subdirectory for the video

    logging.info(f"Processing video: {video_name}")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        logging.error(f"Failed to open video: {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)  # Frame rate
    frame_count = 0
    saved_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            logging.info(f"End of video reached: {video_path}")
            break

        frame_count += 1
        if frame_count % frame_step != 0:
            continue

        # Get timestamp in milliseconds
        timestamp_ms = int(cap.get(cv2.CAP_PROP_POS_MSEC))  # Get timestamp in milliseconds

        # Save frame with timestamp
        frame_filename = os.path.join(video_frames_dir, f"frame_{timestamp_ms:012d}.jpg")
        cv2.imwrite(frame_filename, frame)
        saved_frames += 1

    cap.release()
    logging.info(f"Saved {saved_frames} frames from video: {video_name}")

def process_all_videos(video_folder, output_dir):
    """
    Processes all videos in a folder and its subfolders, extracting frames.

    Args:
        video_folder (str): Directory containing all videos.
        output_dir (str): Directory to save extracted frames.
    """
    logging.info(f"Looking for videos in folder: {video_folder}")
    files_found = False
    for root, dirs, files in os.walk(video_folder):
        for file in files:
            if any(file.endswith(ext) for ext in video_extensions):
                files_found = True
                video_path = os.path.join(root, file)
                logging.info(f"Processing video: {file} in {root}")
                extract_frames(video_path, output_dir)

    if not files_found:
        logging.warning(f"No video files found in {video_folder} or its subfolders.")

if __name__ == "__main__":
    logging.info("Starting frame extraction process...")
    process_all_videos(video_folder, extracted_frames_dir)
    logging.info("Frame extraction process completed.")



In [ ]:
#conver to greyscale
import os
import cv2
import glob
import logging

# Directories
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'  # Replace with your actual path
extracted_frames_dir = os.path.join(parent_directory_path, 'extracted_frames')
extracted_frames_greyscale_dir = os.path.join(parent_directory_path, 'extracted_frames_greyscale')

# Ensure output directory exists
os.makedirs(extracted_frames_greyscale_dir, exist_ok=True)

# Logging configuration
logging.basicConfig(
    filename=os.path.join(parent_directory_path, 'greyscale_conversion.log'),
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def convert_to_greyscale(input_dir, output_dir):
    """
    Converts all images in a directory and its subfolders to greyscale using glob.
    """
    logging.info(f"Looking for images in folder: {input_dir}")

    # Use glob to find all image files (adjust the pattern if needed)
    image_files = glob.glob(os.path.join(input_dir, '**', 'frame_*.jpg'), recursive=True)

    for image_path in image_files:
        try:
            # Read the image
            img = cv2.imread(image_path)
            if img is None:
                logging.warning(f"Failed to read image: {image_path}. Skipping.")
                continue

            # Convert to greyscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Construct the output path using string manipulation
            relative_path = os.path.relpath(image_path, input_dir)
            output_image_path = os.path.join(output_dir, relative_path)
            output_dir_name = os.path.dirname(output_image_path)
            os.makedirs(output_dir_name, exist_ok=True)  # Create output directory

            # Save the greyscale image
            cv2.imwrite(output_image_path, gray_img)
            logging.info(f"Converted {image_path} to greyscale and saved to {output_image_path}")

        except Exception as e:
            logging.exception(f"Error processing {image_path}: {e}")

if __name__ == "__main__":
    logging.info("Starting greyscale conversion process...")
    convert_to_greyscale(extracted_frames_dir, extracted_frames_greyscale_dir)
    logging.info("Greyscale conversion process completed.")

In [ ]:
#lukas kanade method
import cv2
import os
import numpy as np
import logging

# Directories
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'
video_folder = os.path.join(parent_directory_path, 'data')  # Path to the "data" folder
extracted_frames_dir = os.path.join(parent_directory_path, 'extracted_frames')
lukas_kanade_dir = os.path.join(parent_directory_path, 'lukas_kanade')

# Create Lukas Kanade subfolders
subfolders = ['328_109', '340_102', '352_103', '352_108', '352_137', '352_132', '352_131', '352_121', '364_303', '364_105', '352_312', '352_138']
for subfolder in subfolders:
    os.makedirs(os.path.join(lukas_kanade_dir, subfolder), exist_ok=True)

# Logging configuration
logging.basicConfig(
    filename=os.path.join(parent_directory_path, 'lukas_kanade.log'),
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def compute_optical_flow(prev_gray, next_gray):
    """
    Compute the optical flow between two consecutive frames using Lucas-Kanade method.

    Args:
        prev_gray (ndarray): Grayscale image of the previous frame.
        next_gray (ndarray): Grayscale image of the current frame.

    Returns:
        flow (ndarray): Optical flow vectors.
    """
    # Parameters for Lucas-Kanade optical flow
    flow = cv2.calcOpticalFlowFarneback(
        prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0
    )
    return flow

def draw_optical_flow(flow, frame, filename):
    """
    Draws the optical flow on the frame and saves the result.

    Args:
        flow (ndarray): Optical flow vectors.
        frame (ndarray): Current frame.
        filename (str): File name to save the frame with flow visualization.
    """
    # Convert flow to polar coordinates
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Create an image to display the flow
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 255
    hsv[..., 0] = angle * 180 / np.pi / 2  # Hue represents direction
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)  # Value represents magnitude

    flow_bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)  # Convert back to BGR for display

    # Save the optical flow image
    cv2.imwrite(filename, flow_bgr)

def apply_lukas_kanade_to_frames(video_frames_dir, lukas_kanade_subfolder):
    """
    Apply Lucas-Kanade optical flow to all extracted frames and save the flow visualization.

    Args:
        video_frames_dir (str): Directory containing extracted frames.
        lukas_kanade_subfolder (str): Subfolder to save optical flow images.
    """
    frames = sorted([f for f in os.listdir(video_frames_dir) if f.endswith('.jpg')])
    prev_frame = None
    prev_gray = None

    for i, frame_filename in enumerate(frames):
        frame_path = os.path.join(video_frames_dir, frame_filename)
        frame = cv2.imread(frame_path)

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            # Compute optical flow between previous and current frame
            flow = compute_optical_flow(prev_gray, gray_frame)

            # Create the filename for saving the optical flow visualization
            flow_filename = os.path.join(lukas_kanade_subfolder, frame_filename)

            # Draw and save optical flow
            draw_optical_flow(flow, frame, flow_filename)

        prev_frame = frame
        prev_gray = gray_frame

def process_all_videos_for_lukas_kanade(extracted_frames_dir, lukas_kanade_dir):
    """
    Process all videos' extracted frames for optical flow and save the results.

    Args:
        extracted_frames_dir (str): Directory containing extracted frames.
        lukas_kanade_dir (str): Directory to save optical flow images.
    """
    logging.info(f"Looking for extracted frames in folder: {extracted_frames_dir}")

    for root, dirs, files in os.walk(extracted_frames_dir):
        for subfolder in subfolders:
            if subfolder in root:
                lukas_kanade_subfolder = os.path.join(lukas_kanade_dir, subfolder)
                video_frames_dir = root  # Current video frames folder
                logging.info(f"Processing frames in {video_frames_dir} for Lucas-Kanade optical flow.")
                apply_lukas_kanade_to_frames(video_frames_dir, lukas_kanade_subfolder)

if __name__ == "__main__":
    logging.info("Starting Lucas-Kanade optical flow processing...")
    process_all_videos_for_lukas_kanade(extracted_frames_dir, lukas_kanade_dir)
    logging.info("Lucas-Kanade optical flow processing completed.")


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Flatten,
    Input,
    Concatenate,
    GlobalAveragePooling2D,
    Conv2D,
    MaxPooling2D
)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define paths
extracted_frames_dir = "/content/drive/My Drive/pour_la_hackathon/MineAI/extracted_frames"
output_csv = "/content/drive/My Drive/pour_la_hackathon/MineAI/site_data.csv"


In [ ]:
# Subfolders (blast sites)
subfolders = ['328_109', '340_102', '352_103', '352_108', '352_137',
              '352_132', '352_131', '352_121', '364_303', '364_105',
              '352_312', '352_138']

# Load CSV data
csv_data = pd.read_csv(output_csv)
print(csv_data)

       Site  Fragment Size Distribution  Visual Fragmentation  \
0   352_108                           1                     1   
1   352_121                           3                     4   
2   328_109                           4                     4   
3   340_102                           3                     3   
4   352_103                           3                     3   
5   352_131                           4                     4   
6   352_132                           1                     3   
7   352_137                           4                     4   
8   352_138                           2                     2   
9   352_312                           3                     3   
10  364_105                           3                     3   
11  364_303                           3                     3   

    Boulders (>1 m)  Fines (<10 mm)  Most Blasts  \
0                 1               3            2   
1                 3               3            0  

In [ ]:
# Prepare data and labels
image_features = []
labels = []

# Image preprocessing parameters
IMG_SIZE = (128, 128)

def preprocess_image(image_path):
    """Read and preprocess a single image."""
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, IMG_SIZE)  # Resize to IMG_SIZE
    img_normalized = img_resized / 255.0  # Normalize pixel values
    return img_normalized

# Process each subfolder
for subfolder in subfolders:
    subfolder_path = os.path.join(extracted_frames_dir, subfolder)
    # Check for corresponding data in the CSV
    site_data = csv_data[csv_data["Site"] == subfolder]
    if site_data.empty:
        print(f"Warning: No CSV data found for {subfolder}. Skipping...")
        continue

    # Get the label (parameter values) for this site
    site_label = site_data.iloc[0, 1:].values.astype(np.float32)

    # List all image files in the subfolder
    for image_file in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_file)
        # Preprocess image
        img_feature = preprocess_image(image_path)
        image_features.append(img_feature)
        labels.append(site_label)


In [ ]:

# Convert to numpy arrays
X = np.array(image_features)
y = np.array(labels)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='linear')  # Output layer with the number of features
])


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ReduceLROnPlateau

# # Image Size (adjust accordingly)
# IMG_SIZE = (128, 128)  # You can adjust the size based on your dataset

# # Build the CNN model with adjustments
# model = Sequential([
#     Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
#     BatchNormalization(),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),

#     Conv2D(128, (3, 3), activation='relu'),
#     BatchNormalization(),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),

#     Conv2D(256, (3, 3), activation='relu'),
#     BatchNormalization(),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),

#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     Dense(y_train.shape[1], activation='linear')  # Output layer with the number of features
# ])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
import os
from datetime import datetime
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError



# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError(), metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Define the output directory
output_dir = "/content/drive/My Drive/pour_la_hackathon/MineAI/output"
os.makedirs(output_dir, exist_ok=True)

# Generate a filename with the current date and time
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_save_path = os.path.join(output_dir, f"blast_analysis_model_{timestamp}.h5")

# Save the model
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# Keep only the latest model
saved_models = sorted(
    [os.path.join(output_dir, file) for file in os.listdir(output_dir) if file.endswith('.h5')],
    key=os.path.getmtime,
    reverse=True
)

# Remove older models, keeping the latest one
for old_model in saved_models[1:]:
    os.remove(old_model)
    print(f"Deleted old model: {old_model}")

print(f"Latest model: {saved_models[0]}")


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 140ms/step - loss: 11.8225 - mae: 2.5314 - val_loss: 5.3045 - val_mae: 1.8518
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 5.1225 - mae: 1.8343 - val_loss: 4.5447 - val_mae: 1.6943
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 4.2187 - mae: 1.6376 - val_loss: 3.9601 - val_mae: 1.5806
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.2503 - mae: 1.4255 - val_loss: 3.8094 - val_mae: 1.5587
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 2.8076 - mae: 1.3136 - val_loss: 3.0024 - val_mae: 1.3796
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 2.5794 - mae: 1.2589 - val_loss: 2.9806 - val_mae: 1.3733
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 2.2717 - mae: 1.1668 - val_loss: 3.2756 - val_mae: 1.4429
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 2.4138 - mae: 1.1984 - val_loss: 2.9546 - val_mae: 1.3626
Epoch 9/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 2.2

Model saved to /content/drive/My Drive/pour_la_hackathon/MineAI/output/blast_analysis_model_20250115_054827.h5
Deleted old model: /content/drive/My Drive/pour_la_hackathon/MineAI/output/blast_analysis_model_20250115_054624.h5
Latest model: /content/drive/My Drive/pour_la_hackathon/MineAI/output/blast_analysis_model_20250115_054827.h5


In [ ]:
#How to increase model performance
# 1. Increase Model Depth:
# Add more convolutional layers with more filters or experiment with different layer architectures.

# 2. Use a Different Optimizer:
# Try optimizers like RMSprop or SGD with momentum, or try fine-tuning the learning rate.

# 3. Data Preprocessing:
# Normalize your data before training (scaling pixel values to [0, 1]).
# Consider using a validation split during training to ensure that the model generalizes better.
# 4. Epoch Adjustment:
# Increase epochs, but use a learning rate scheduler to adjust learning rates as training progresses to improve convergence.

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])

# Use a learning rate scheduler to reduce the learning rate if the model stops improving
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the model with augmented data
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=50,  # Increase epochs for more training
    callbacks=[lr_scheduler]
)

# Evaluate the model
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Calculate accuracy manually for multi-output
# Round the predictions and ground truth
y_pred_rounded = np.rint(y_pred).astype(int)
y_test_rounded = np.rint(y_test).astype(int)

# Calculate accuracy for each output individually
accuracies = []
for i in range(y_pred_rounded.shape[1]):  # For each output (column)
    accuracies.append(np.mean(y_pred_rounded[:, i] == y_test_rounded[:, i]))

# Calculate the average accuracy across all outputs
average_accuracy = np.mean(accuracies)

# Print evaluation metrics
print(f"Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R2 Score): {r2:.4f}")
print(f"Accuracy (Average across outputs): {average_accuracy:.4f}")

# Save the model as before
output_dir = "/content/drive/My Drive/pour_la_hackathon/MineAI/output"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_save_path = os.path.join(output_dir, f"blast_analysis_model_{timestamp}.h5")

# Save the model
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# Keep only the latest model
saved_models = sorted(
    [os.path.join(output_dir, file) for file in os.listdir(output_dir) if file.endswith('.h5')],
    key=os.path.getmtime,
    reverse=True
)

# Remove older models, keeping the latest one
for old_model in saved_models[1:]:
    os.remove(old_model)
    print(f"Deleted old model: {old_model}")

print(f"Latest model: {saved_models[0]}")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 290ms/step - loss: 3.2624 - mae: 1.3906 - val_loss: 1.5648 - val_mae: 0.9436 - learning_rate: 1.0000e-04
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - loss: 2.2527 - mae: 1.1506 - val_loss: 1.6671 - val_mae: 0.9941 - learning_rate: 1.0000e-04
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 2.1946 - mae: 1.1500 - val_loss: 1.8135 - val_mae: 1.0502 - learning_rate: 1.0000e-04
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 103ms/step - loss: 2.2359 - mae: 1.1730 - val_loss: 1.7361 - val_mae: 1.0259 - learning_rate: 1.0000e-04
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - loss: 2.1627 - mae: 1.1428 - val_loss: 1.7383 - val_mae: 1.0256 - learning_rate: 5.0000e-05
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - loss: 2.1833 - mae: 1.1475 - val_loss: 1.7815 - val_mae: 1.0384 - learning_rate: 5.0000e-05
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 109ms/step - loss: 2.1817 - mae: 1.1534 - val_loss: 1.7307 - val_mae: 1.0218 - learning_rate: 5.

Evaluation Metrics:
Mean Absolute Error (MAE): 1.0346
Mean Squared Error (MSE): 1.7666
R-squared (R2 Score): -0.2337
Accuracy (Average across outputs): 0.3247
Model saved to /content/drive/My Drive/pour_la_hackathon/MineAI/output/blast_analysis_model_20250115_055159.h5
Deleted old model: /content/drive/My Drive/pour_la_hackathon/MineAI/output/blast_analysis_model_20250115_054827.h5
Latest model: /content/drive/My Drive/pour_la_hackathon/MineAI/output/blast_analysis_model_20250115_055159.h5


In [33]:
import numpy as np
import os
import tensorflow as tf
import cv2

from google.colab import drive
drive.flush_and_unmount()


#importing the given data from google drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the file in Google Drive
parent_directory_path = '/content/drive/My Drive/pour_la_hackathon/MineAI'

# Define image size for input shape
IMG_SIZE = (128, 128)  # Example image size, adjust accordingly

def preprocess_image(image_path):
    """Read and preprocess a single image."""
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, IMG_SIZE)  # Resize to IMG_SIZE
    img_normalized = img_resized / 255.0  # Normalize pixel values
    return img_normalized

def predict_new_blast(model_path, new_frames_dir):
    """Predict features for a new set of frames."""
    # Load the trained model
    model = tf.keras.models.load_model(model_path)

    # Process new frames
    new_features = []
    frame_predictions = []  # To store individual frame predictions
    image_files = os.listdir(new_frames_dir)

    for image_file in image_files:
        image_path = os.path.join(new_frames_dir, image_file)
        img_feature = preprocess_image(image_path)
        new_features.append(img_feature)

        # Predict for the current frame
        prediction = model.predict(np.expand_dims(img_feature, axis=0))  # Single frame prediction
        frame_predictions.append(prediction[0])  # Store prediction for this frame

    # Convert to numpy array and predict
    new_features = np.array(new_features)
    aggregated_prediction = np.mean(frame_predictions, axis=0)

    return aggregated_prediction, frame_predictions, len(image_files)

def map_predictions_to_grades(predictions):
    """Map predicted features to grades based on conditions."""
    grades = []
    for pred in predictions:
        if pred > 3.5:
            grade = 4
        elif 2.5 <= pred <= 3.5:
            grade = 3
        elif 1.5 <= pred <= 2.5:
            grade = 2
        elif 0.5 <= pred <= 1.5:  # Adjusted lower bound
            grade = 1
        else:
            grade = 0
        grades.append(grade)
    return grades


# Example usage
new_frames_dir = "/content/drive/My Drive/pour_la_hackathon/MineAI/unknown_blast_frames"
model_save_path = "/content/drive/My Drive/pour_la_hackathon/MineAI/blast_analysis_model.h5"

# Predict the features for the new blast frames
predicted_features, individual_predictions, num_images = predict_new_blast(model_save_path, new_frames_dir)

# Display number of images and individual predictions
# print(f"Number of images processed: {num_images}")
# print(f"Individual frame predictions:")
# for i, pred in enumerate(individual_predictions):
#     print(f"Frame {i+1}: {pred}")

# Map the predicted features to grades based on the conditions
final_grades = map_predictions_to_grades(predicted_features)
print("Mapped grades for the new blast:", final_grades)


from tabulate import tabulate

# Column labels
columns = [
    "Fragment Size Distribution", "Visual Fragmentation", "Boulders (>1 m)", "Fines (<10 mm)",
    "Most Blasts", "For Development and Pioneering Blast", "Digability (Ease of Excavation) - Simple Method",
    "Muckpile Height", "Swell Factor", "Fragmentation Distribution", "Muckpile Shape",
    "Blast Sequence", "Time Gap/Overlap", "Muckpile Throw", "Misfires/Blowouts", "Swell and Spread",
    "Fragmentation Distribution", "Muckpile Shape", "Misfires/Blowouts",
    "Confinement and Ejection (Stemming Quality)", "(Vibration Monitor data available) Fish Spawning season - Yes",
    "(Vibration Monitor data available) Fish Spawning season - No", "Vibration Monitor Data not available"
]

# Grades for each column

# Combine column labels and grades into a single table row
table_data = [final_grades]

# Display the table
print(tabulate(table_data, headers=columns, tablefmt="grid"))


# Define the ranges for averaging
ranges = [
    (0, 3),  # arr[0] to arr[3] is for fragmentation quality
    (4, 5),  # arr[4] and arr[5] is for flyrock safety
    (6, 10), # arr[6] to arr[10] is for digability
    (11, 14),# arr[11] to arr[14] is for blast timing
    (15, 18),# arr[15] to arr[18] is for powder factor and energy distribution
    (19, 19),# arr[19] only is for confinement and ejection
    (20, 22) # arr[20] to arr[22] is for vibration and overpressure
]



# Function to calculate average ignoring zeros
def average_ignore_zeros(arr):
    non_zero_elements = [x for x in arr if x != 0]
    if not non_zero_elements:  # If all elements are zero
        return 0
    return sum(non_zero_elements) / len(non_zero_elements)


# Calculate averages for the specified ranges
averages = []
for start, end in ranges:
    subset = final_grades[start:end+1]
    avg = average_ignore_zeros(subset)
    averages.append(avg)

# Output the averages
# for i, avg in enumerate(averages, start=1):
    # print(f"Average for range {ranges[i-1]}: {avg:.2f}")

# Output the approximated averages
for i, avg in enumerate(averages, start=1):
    approximated_avg = round(avg)  # Round the average to the nearest integer
    # print(f"Average for range (rounded off) {ranges[i-1]}: {approximated_avg}")

# Output the approximated averages and store them
approximated_averages = []

for i, avg in enumerate(averages, start=1):
    approximated_avg = round(avg)  # Round the average to the nearest integer
    approximated_averages.append(approximated_avg)
    print(f"Average for range {ranges[i-1]}: {approximated_avg}")

# Compute the mean of the rounded averages (ignoring 0)
non_zero_approximated_averages = [x for x in approximated_averages if x != 0]
overall_mean = sum(non_zero_approximated_averages) / len(non_zero_approximated_averages)

# Round the overall mean
final_mean = round(overall_mean)
# print(f"Overall mean of rounded averages: {final_mean}")

# Define grade mapping
grade_mapping = {4: 'A', 3: 'B', 2: 'C', 1: 'F'}

# Get the final letter grade
final_letter_grade = grade_mapping.get(final_mean, 'Invalid Grade')
print(f"Final grade of the blast: {final_letter_grade}")

# Define the ranges for averaging
ranges = [
    (0, 3),  # arr[0] to arr[3] is for fragmentation quality
    (4, 5),  # arr[4] and arr[5] is for flyrock safety
    (6, 10), # arr[6] to arr[10] is for digability
    (11, 14),# arr[11] to arr[14] is for blast timing
    (15, 18),# arr[15] to arr[18] is for powder factor and energy distribution
    (19, 19),# arr[19] only is for confinement and ejection
    (20, 22) # arr[20] to arr[22] is for vibration and overpressure
]

# Function to calculate average ignoring zeros
def average_ignore_zeros(arr):
    non_zero_elements = [x for x in arr if x != 0]
    if not non_zero_elements:  # If all elements are zero
        return 0
    return sum(non_zero_elements) / len(non_zero_elements)

# Calculate averages for the specified ranges
averages = []
for start, end in ranges:
    subset = final_grades[start:end+1]
    avg = average_ignore_zeros(subset)
    averages.append(avg)

# Output the approximated averages and convert to grades
approximated_averages = []
hashmap = {
    "Fragmentation Quality": None,
    "Flyrock and Safety": None,
    "Digability": None,
    "Blast Timing": None,
    "Powder Factor and Energy Distribution": None,
    "Confinement and Ejection": None,
    "Vibration and Overpressure": None
}

# Define grade mapping
grade_mapping = {4: 'A', 3: 'B', 2: 'C', 1: 'F'}

for i, avg in enumerate(averages, start=1):
    approximated_avg = round(avg)  # Round the average to the nearest integer
    approximated_averages.append(approximated_avg)

    # Convert the approximated average to grade
    grade = grade_mapping.get(approximated_avg, 'Invalid Grade')

    key = list(hashmap.keys())[i - 1]
    hashmap[key] = grade

# Compute the mean of the rounded averages (ignoring 0)
non_zero_approximated_averages = [x for x in approximated_averages if x != 0]
overall_mean = sum(non_zero_approximated_averages) / len(non_zero_approximated_averages)

# Round the overall mean
final_mean = round(overall_mean)

# Convert the final mean to a grade
final_letter_grade = grade_mapping.get(final_mean, 'Invalid Grade')


# print(f"\nFinal grade of the blast: {final_letter_grade}")

# Corrected recommendations dictionary with only relevant keys
recommendations = {
    "Fragmentation Quality": {
        "A": "No improvement needed. The blast is well-executed, and fragmentation is ideal for excavation.",
        "B": "Increase explosive energy: Consider increasing the Powder Factor (PF) or using a more energetic explosive to achieve better fragmentation. Optimize hole diameter and adjust timing sequence.",
        "C": "Increase explosive charge and burden. Use higher-energy explosives or boosters to improve fragmentation.",
        "F": "Increase charge, reduce hole spacing, change detonation sequence, and optimize stemming."
    },
    "Flyrock and Safety": {
        "A": "No improvement needed. The blast was executed safely with controlled flyrock.",
        "B": "Increase blast matting, expand safety zone, and reduce PF to prevent over-breaking of the rock.",
        "C": "Increase stemming height, improve initiation sequence, and increase burden.",
        "F": "Increase safety zone, use blast barriers/mats, reassess explosive charge, and improve initiation design."
    },
    "Digability": {
        "A": "No improvement needed. The blast has achieved ideal material movement and fragmentation.",
        "B": "Optimize blast timing, adjust stemming height.",
        "C": "Increase explosive charge, increase burden/spacing for energy distribution.",
        "F": "Increase explosive energy, use delay blasting, and increase blast diameter."
    },
    "Vibration and Overpressure": {
        "A": "No improvement needed. The blast was within safe vibration limits.",
        "B": "Increase delay times, reduce explosive charge.",
        "C": "Adjust timing, increase stemming height.",
        "F": "Revisit blast design, add blast mats, reassess explosive type."
    },
    "Timing and Sequencing": {
        "A": "No improvement needed. The blast was timed and sequenced well.",
        "B": "Fine-tune delay sequences.",
        "C": "Adjust timing significantly, minimize gaps in detonation sequence.",
        "F": "Rework the entire timing and sequencing strategy."
    }
}

# Define the ranges for averaging (only the relevant ranges are included)
ranges = [
    (0, 3),  # Fragmentation Quality
    (4, 5),  # Flyrock and Safety
    (6, 10), # Digability
    (11, 14),# Blast Timing
    (20, 22) # Vibration and Overpressure
]

# Function to calculate average ignoring zeros
def average_ignore_zeros(arr):
    non_zero_elements = [x for x in arr if x != 0]
    if not non_zero_elements:  # If all elements are zero
        return 0
    return sum(non_zero_elements) / len(non_zero_elements)

# Calculate averages for the specified ranges
averages = []
for start, end in ranges:
    subset = final_grades[start:end+1]
    avg = average_ignore_zeros(subset)
    averages.append(avg)

# Output the approximated averages and convert to grades
approximated_averages = []
hashmap = {
    "Fragmentation Quality": None,
    "Flyrock and Safety": None,
    "Digability": None,
    "Timing and Sequencing": None,
    "Vibration and Overpressure": None
}

# Define grade mapping
grade_mapping = {4: 'A', 3: 'B', 2: 'C', 1: 'F'}

# Populate the hashmap with grades and recommendations
for i, avg in enumerate(averages, start=1):
    approximated_avg = round(avg)  # Round the average to the nearest integer
    approximated_averages.append(approximated_avg)

    grade = grade_mapping.get(approximated_avg, 'Invalid Grade')
    key = list(hashmap.keys())[i - 1]
    hashmap[key] = grade

# Generate the scorecard with recommendations
scorecard = {}
for parameter, grade in hashmap.items():
    scorecard[parameter] = {
        "Grade": grade,
        "Recommendation": recommendations[parameter].get(grade, 'No recommendation available')
    }

# Output the scorecard
print("Blast Quality Scorecard:")
for parameter, details in scorecard.items():
    print(f"\n{parameter}:")
    print(f"  Grade: {details['Grade']}")
    print(f"  Recommendation: {details['Recommendation']}")



Mounted at /content/drive


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━


Final grade of the blast: B
Blast Quality Scorecard:

Fragmentation Quality:
  Grade: B
  Recommendation: Increase explosive energy: Consider increasing the Powder Factor (PF) or using a more energetic explosive to achieve better fragmentation. Optimize hole diameter and adjust timing sequence.

Flyrock and Safety:
  Grade: B
  Recommendation: Increase blast matting, expand safety zone, and reduce PF to prevent over-breaking of the rock.

Digability:
  Grade: F
  Recommendation: Increase explosive energy, use delay blasting, and increase blast diameter.

Timing and Sequencing:
  Grade: B
  Recommendation: Fine-tune delay sequences.

Vibration and Overpressure:
  Grade: B
  Recommendation: Increase delay times, reduce explosive charge.
